# Evaluate finetuned Mistral Models

## Imports and environment variables

In [ ]:
import torch
import os
import sys
from dotenv import load_dotenv

from peft import PeftModel

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, AutoModelForCausalLM

from rl_training_new.utils import find_best_window

# Load environment variables from .env file
load_dotenv()

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..')))

# load the relevant devices available on the server
os.environ["CUDA_VISIBLE_DEVICES"] = os.getenv("AVAILABLE_DEVICES")

# Enable expandable CUDA segments
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# load cuda
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print("CUDA is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")


In [ ]:
MODEL = os.getenv("GENERATION_MODEL_NAME")
ALGORITHM = os.getenv("EVAL_MODEL_ALGORITHM")
RL_TRAINED_ADAPTERS = os.getenv("EVAL_MODEL_FOLDER")

## Load model

In [ ]:
base_model_test = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=1)
new_model = PeftModel.from_pretrained(base_model_test, RL_TRAINED_ADAPTERS)

## ROUGE/BLEU on relevant sequence from the answer 

In [ ]:
# Evaluation metric 1: ROUGE on relevant sequences
# WHy not BLEU --> penalizes missing ngrams, not something I m looking for here







## BERT Score --> based on embedding similarity

In [ ]:
# Maybe add BERTScore --> semantic similarity based on sentencetransformer

## G-Eval

In [ ]:
# Evaluation metric 2: Evaluate whole answer on G-Eval

## Personal evaluation

In [ ]:
# Generate about 10 answers with both models --> load them into the user interface and get reward score
# Also maybe do qualitative evaluation